In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="bge-m3:567m",
)

In [3]:
# 載入 PDF 文件
loader = PyPDFLoader('./data/PDF_file.pdf')
docs = loader.load_and_split()

chunk_size = 256
chunk_overlap = 128

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
documents = text_splitter.split_documents(docs)

# 將文檔寫入 ChromaDB
db = Chroma.from_documents(
    documents,
    embedding=embeddings_model,
    persist_directory="./story-db" # bge-m3:567m
)

In [9]:
# Setup the Retriever
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

ollama_llm = OllamaLLM(
    base_url='http://dandelion-ollama-1:11434', 
    model="llama3.1:8b",
    temperature=0.0,
    num_predict=512
)

# 建立 RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    retriever=db.as_retriever(),
)

# 提問
query = "玫瑰是誰？"
result = qa_chain.invoke(query)
# print('提問:', query)
print("\n回答: ")
print(result)



回答: 
{'query': '玫瑰是誰？', 'result': '玫瑰是小王子的玫瑰花，還有園中其他五千朵玫瑰花（但小王子的玫瑰花是獨一無二的）。'}


In [10]:
qa_chain.invoke('最令你印象深刻的小王子的經歷是？')

{'query': '最令你印象深刻的小王子的經歷是？',
 'result': '最令我印象深刻的小王子的經歷是他無意中吐露的一些話逐漸使我搞清了他的來歷。'}

In [11]:
qa_chain.invoke('小王子的來歷是什？')

{'query': '小王子的來歷是什？', 'result': '小王子所來自的那個星球是小行星B612。'}

In [12]:
qa_chain.invoke('你是哪個模型？')

{'query': '你是哪個模型？', 'result': '我不知道。'}

In [15]:
qa_chain.invoke('誰喜歡喝酒？他出現在文章中哪裡？')

{'query': '誰喜歡喝酒？他出現在文章中哪裡？', 'result': '酒鬼。它出現在第十二段（XII）中。'}